<a href="https://colab.research.google.com/github/Dinesh-Adhithya-H/Deep_Learning_using_Tensorflow/blob/main/C7_2_Transfer_Learning_ResNet50.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.applications.resnet50 import ResNet50
import tensorflow_datasets as tfds

In [2]:
batch_size=32
classes=["airplane","automobile","bird","cat",'deer', 'dog',"frog"'horse', 'ship', 'truck']

In [3]:
(train_images,train_labels),(validation_images,validation_labels)=tf.keras.datasets.cifar10.load_data()

170500096/170498071 [==============================] - 3s 0us/step


In [4]:
def preprocess(images):
  images=images.astype("float32")
  output=tf.keras.applications.resnet50.preprocess_input(images)
  return output

In [5]:
train_x=preprocess(train_images)
valid_x=preprocess(validation_images)

In [6]:
def feature_extract(inputs):
  output=tf.keras.applications.resnet.ResNet50(input_shape=(224,224,3),include_top=False,weights='imagenet')(inputs)
  return output

In [7]:
def classifier(inputs):
  x=tf.keras.layers.GlobalAveragePooling2D()(inputs)
  x=tf.keras.layers.Flatten()(x)
  x=tf.keras.layers.Dense(1024,activation="relu")(x)
  x=tf.keras.layers.Dense(512,activation="relu")(x)
  x=tf.keras.layers.Dense(10,activation="softmax",name="classification")(x)
  return x

In [8]:
def final_model(inputs):
  resize=tf.keras.layers.UpSampling2D(size=(7,7))(inputs)
  resnet=feature_extract(resize)
  class_output=classifier(resnet)
  return class_output

In [9]:
def compile_model():
  inputs=tf.keras.layers.Input(shape=(32,32,3))
  classification=final_model(inputs)
  model=tf.keras.Model(inputs=inputs,outputs=classification)
  model.compile(optimizer="SGD",loss="sparse_categorical_crossentropy",metrics=["accuracy"])
  return model

In [10]:
model=compile_model()
model.summary()

94773248/94765736 [==============================] - 1s 0us/step
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
up_sampling2d (UpSampling2D) (None, 224, 224, 3)       0         
_________________________________________________________________
resnet50 (Functional)        (None, 7, 7, 2048)        23587712  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
flatten (Flatten)            (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 1024)              2098176   
______________________________________________________________

In [12]:
eph=10
history=model.fit(train_x,train_labels,epochs=eph,validation_data=(valid_x,validation_labels),batch_size=20)

Epoch 1/10
 200/2500 [=>............................] - ETA: 8:34:11 - loss: 0.9488 - accuracy: 0.6825

KeyboardInterrupt: ignored

In [13]:
model.evaluate(valid_x,validation_labels)

 41/313 [==>...........................] - ETA: 21:52 - loss: 0.5670 - accuracy: 0.8224

KeyboardInterrupt: ignored